I guess most people prefer gradient boosting models to neural networks in this conpetition. But I don't have enough domain knowledge, I relied on the neural network model first. In my model, the additional data(mulliken charge and tensors) are used as the output of the middle of neural network model. It slightly improved the score.

This is my first kernel and I am a beginner of both data science and programming. Any comments and advices are welcome!

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.layers import Dense, Input, Activation
from keras.layers import BatchNormalization,Add
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)


Using TensorFlow backend.


## Load files
load the additional data as well.

In [2]:
df_train=pd.read_csv('../input/train.csv')
df_test=pd.read_csv('../input/test.csv')
df_struct=pd.read_csv('../input/structures.csv')

#df_train_sub_potential=pd.read_csv('../input/potential_energy.csv')
#df_train_sub_moment=pd.read_csv('../input/dipole_moments.csv')
df_train_sub_charge=pd.read_csv('../input/mulliken_charges.csv')
df_train_sub_tensor=pd.read_csv('../input/magnetic_shielding_tensors.csv')


## Feature Engineering
Let's merge all data and create features. The additional features are only used for training session.It will come up later.

I use this great kernel. 
https://www.kaggle.com/seriousran/just-speed-up-calculate-distance-from-benchmark

In [3]:
def map_atom_info(df_1,df_2, atom_idx):
    df = pd.merge(df_1, df_2, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    df = df.drop('atom_index', axis=1)

    return df



for atom_idx in [0,1]:
    df_train = map_atom_info(df_train,df_struct, atom_idx)
    df_train = map_atom_info(df_train,df_train_sub_charge, atom_idx)
    df_train = map_atom_info(df_train,df_train_sub_tensor, atom_idx)
    df_train = df_train.rename(columns={'atom': f'atom_{atom_idx}',
                                        'x': f'x_{atom_idx}',
                                        'y': f'y_{atom_idx}',
                                        'z': f'z_{atom_idx}',
                                        'mulliken_charge': f'charge_{atom_idx}',
                                        'XX': f'XX_{atom_idx}',
                                        'YX': f'YX_{atom_idx}',
                                        'ZX': f'ZX_{atom_idx}',
                                        'XY': f'XY_{atom_idx}',
                                        'YY': f'YY_{atom_idx}',
                                        'ZY': f'ZY_{atom_idx}',
                                        'XZ': f'XZ_{atom_idx}',
                                        'YZ': f'YZ_{atom_idx}',
                                        'ZZ': f'ZZ_{atom_idx}',})
    df_test = map_atom_info(df_test,df_struct, atom_idx)
    df_test = df_test.rename(columns={'atom': f'atom_{atom_idx}',
                                'x': f'x_{atom_idx}',
                                'y': f'y_{atom_idx}',
                                'z': f'z_{atom_idx}'})
    #add some features
    df_struct['c_x']=df_struct.groupby('molecule_name')['x'].transform('mean')
    df_struct['c_y']=df_struct.groupby('molecule_name')['y'].transform('mean')
    df_struct['c_z']=df_struct.groupby('molecule_name')['z'].transform('mean')
    df_struct['atom_n']=df_struct.groupby('molecule_name')['atom_index'].transform('max')



Distance is very effective feature.

In [5]:
def make_features(df):
    df['dx']=df['x_1']-df['x_0']
    df['dy']=df['y_1']-df['y_0']
    df['dz']=df['z_1']-df['z_0']
    df['distance']=(df['dx']**2+df['dy']**2+df['dz']**2)**(1/2)
    return df
df_train=make_features(df_train)
df_test=make_features(df_test) 

In [ ]:
Cosine angles also works.
https://www.kaggle.com/kmat2019/effective-feature

In [6]:
def get_closest_farthest(df):
    df_temp=df.loc[:,["molecule_name","atom_index_0","atom_index_1","distance","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()
    df_temp_=df_temp.copy()
    df_temp_= df_temp_.rename(columns={'atom_index_0': 'atom_index_1',
                                       'atom_index_1': 'atom_index_0',
                                       'x_0': 'x_1',
                                       'y_0': 'y_1',
                                       'z_0': 'z_1',
                                       'x_1': 'x_0',
                                       'y_1': 'y_0',
                                       'z_1': 'z_0'})
    df_temp_all=pd.concat((df_temp,df_temp_),axis=0)

    df_temp_all["min_distance"]=df_temp_all.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('min')
    df_temp_all["max_distance"]=df_temp_all.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('max')
    
    df_temp= df_temp_all[df_temp_all["min_distance"]==df_temp_all["distance"]].copy()
    df_temp=df_temp.drop(['x_0','y_0','z_0','min_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_closest',
                                         'distance': 'distance_closest',
                                         'x_1': 'x_closest',
                                         'y_1': 'y_closest',
                                         'z_1': 'z_closest'})

    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_closest': f'atom_index_closest_{atom_idx}',
                                        'distance_closest': f'distance_closest_{atom_idx}',
                                        'x_closest': f'x_closest_{atom_idx}',
                                        'y_closest': f'y_closest_{atom_idx}',
                                        'z_closest': f'z_closest_{atom_idx}'})

    df_temp= df_temp_all[df_temp_all["max_distance"]==df_temp_all["distance"]].copy()
    df_temp=df_temp.drop(['x_0','y_0','z_0','max_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_farthest',
                                         'distance': 'distance_farthest',
                                         'x_1': 'x_farthest',
                                         'y_1': 'y_farthest',
                                         'z_1': 'z_farthest'})

    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_farthest': f'atom_index_farthest_{atom_idx}',
                                        'distance_farthest': f'distance_farthest_{atom_idx}',
                                        'x_farthest': f'x_farthest_{atom_idx}',
                                        'y_farthest': f'y_farthest_{atom_idx}',
                                        'z_farthest': f'z_farthest_{atom_idx}'})
    return df
    
df_train=get_closest_farthest(df_train)    
df_test=get_closest_farthest(df_test)    

In [8]:
def add_cos_features(df):
    df["distance_center0"]=((df['x_0']-df['c_x'])**2+(df['y_0']-df['c_y'])**2+(df['z_0']-df['c_z'])**2)**(1/2)
    df["distance_center1"]=((df['x_0']-df['c_x'])**2+(df['y_0']-df['c_y'])**2+(df['z_0']-df['c_z'])**2)**(1/2)
    df["distance_c0"]=((df['x_0']-df['x_closest_0'])**2+(df['y_0']-df['y_closest_0'])**2+(df['z_0']-df['z_closest_0'])**2)**(1/2)
    df["distance_c1"]=((df['x_1']-df['x_closest_1'])**2+(df['y_1']-df['y_closest_1'])**2+(df['z_1']-df['z_closest_1'])**2)**(1/2)
    df["distance_f0"]=((df['x_0']-df['x_farthest_0'])**2+(df['y_0']-df['y_farthest_0'])**2+(df['z_0']-df['z_farthest_0'])**2)**(1/2)
    df["distance_f1"]=((df['x_1']-df['x_farthest_1'])**2+(df['y_1']-df['y_farthest_1'])**2+(df['z_1']-df['z_farthest_1'])**2)**(1/2)
    df["vec_center0_x"]=(df['x_0']-df['c_x'])/(df["distance_center0"]+1e-10)
    df["vec_center0_y"]=(df['y_0']-df['c_y'])/(df["distance_center0"]+1e-10)
    df["vec_center0_z"]=(df['z_0']-df['c_z'])/(df["distance_center0"]+1e-10)
    df["vec_center1_x"]=(df['x_1']-df['c_x'])/(df["distance_center1"]+1e-10)
    df["vec_center1_y"]=(df['y_1']-df['c_y'])/(df["distance_center1"]+1e-10)
    df["vec_center1_z"]=(df['z_1']-df['c_z'])/(df["distance_center1"]+1e-10)
    df["vec_c0_x"]=(df['x_0']-df['x_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c0_y"]=(df['y_0']-df['y_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c0_z"]=(df['z_0']-df['z_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c1_x"]=(df['x_1']-df['x_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_c1_y"]=(df['y_1']-df['y_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_c1_z"]=(df['z_1']-df['z_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_f0_x"]=(df['x_0']-df['x_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f0_y"]=(df['y_0']-df['y_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f0_z"]=(df['z_0']-df['z_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f1_x"]=(df['x_1']-df['x_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_f1_y"]=(df['y_1']-df['y_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_f1_z"]=(df['z_1']-df['z_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_x"]=(df['x_1']-df['x_0'])/df["distance"]
    df["vec_y"]=(df['y_1']-df['y_0'])/df["distance"]
    df["vec_z"]=(df['z_1']-df['z_0'])/df["distance"]
    df["cos_c0_c1"]=df["vec_c0_x"]*df["vec_c1_x"]+df["vec_c0_y"]*df["vec_c1_y"]+df["vec_c0_z"]*df["vec_c1_z"]
    df["cos_f0_f1"]=df["vec_f0_x"]*df["vec_f1_x"]+df["vec_f0_y"]*df["vec_f1_y"]+df["vec_f0_z"]*df["vec_f1_z"]
    df["cos_center0_center1"]=df["vec_center0_x"]*df["vec_center1_x"]+df["vec_center0_y"]*df["vec_center1_y"]+df["vec_center0_z"]*df["vec_center1_z"]
    df["cos_c0"]=df["vec_c0_x"]*df["vec_x"]+df["vec_c0_y"]*df["vec_y"]+df["vec_c0_z"]*df["vec_z"]
    df["cos_c1"]=df["vec_c1_x"]*df["vec_x"]+df["vec_c1_y"]*df["vec_y"]+df["vec_c1_z"]*df["vec_z"]
    df["cos_f0"]=df["vec_f0_x"]*df["vec_x"]+df["vec_f0_y"]*df["vec_y"]+df["vec_f0_z"]*df["vec_z"]
    df["cos_f1"]=df["vec_f1_x"]*df["vec_x"]+df["vec_f1_y"]*df["vec_y"]+df["vec_f1_z"]*df["vec_z"]
    df["cos_center0"]=df["vec_center0_x"]*df["vec_x"]+df["vec_center0_y"]*df["vec_y"]+df["vec_center0_z"]*df["vec_z"]
    df["cos_center1"]=df["vec_center1_x"]*df["vec_x"]+df["vec_center1_y"]*df["vec_y"]+df["vec_center1_z"]*df["vec_z"]
    df=df.drop(['vec_c0_x','vec_c0_y','vec_c0_z','vec_c1_x','vec_c1_y','vec_c1_z',
                'vec_f0_x','vec_f0_y','vec_f0_z','vec_f1_x','vec_f1_y','vec_f1_z',
                'vec_center0_x','vec_center0_y','vec_center0_z','vec_center1_x','vec_center1_y','vec_center1_z',
                'vec_x','vec_y','vec_z'], axis=1)
    return df
    
df_train=add_cos_features(df_train)
df_test=add_cos_features(df_test)

In [9]:
df_train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,charge_0,XX_0,YX_0,ZX_0,XY_0,YY_0,ZY_0,XZ_0,YZ_0,ZZ_0,atom_1,x_1,y_1,z_1,c_x,c_y,c_z,atom_n,charge_1,XX_1,YX_1,ZX_1,XY_1,YY_1,ZY_1,XZ_1,YZ_1,ZZ_1,dx,dy,...,distance,atom_index_closest_0,distance_closest_0,x_closest_0,y_closest_0,z_closest_0,max_distance_x,atom_index_closest_1,distance_closest_1,x_closest_1,y_closest_1,z_closest_1,max_distance_y,atom_index_farthest_0,distance_farthest_0,x_farthest_0,y_farthest_0,z_farthest_0,min_distance_x,atom_index_farthest_1,distance_farthest_1,x_farthest_1,y_farthest_1,z_farthest_1,min_distance_y,distance_center0,distance_center1,distance_c0,distance_c1,distance_f0,distance_f1,cos_c0_c1,cos_f0_f1,cos_center0_center1,cos_c0,cos_c1,cos_f0,cos_f1,cos_center0,cos_center1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,0.133921,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861,C,-0.012698,1.085804,0.008001,-0.012689,1.085797,0.008001,4,-0.535689,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,-0.014849,1.091835,...,1.091953,0,1.091953,-0.012698,1.085804,0.008001,1.783157,3,1.091946,-0.540815,1.447527,-0.876644,1.091953,4,1.783157,-0.523814,1.437933,0.906397,1.091953,1,1.091953,0.002150,-0.006031,0.001976,1.091946,1.091945,1.091945,1.091953,1.091946,1.783157,1.091953,0.333335,-0.816502,-0.000007,-1.000000,-0.333335,-0.816502,1.000000,-1.000000,0.000007
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,0.133921,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861,H,1.011731,1.463751,0.000277,-0.012689,1.085797,0.008001,4,0.133922,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967,1.009580,1.469782,...,1.783120,0,1.091953,-0.012698,1.085804,0.008001,1.783157,0,1.091952,-0.012698,1.085804,0.008001,1.783158,4,1.783157,-0.523814,1.437933,0.906397,1.091953,3,1.783158,-0.540815,1.447527,-0.876644,1.091952,1.091945,1.091945,1.091953,1.091952,1.783157,1.783158,-0.333287,0.000016,-0.333304,-0.816483,0.816482,-0.499994,0.499995,-0.816487,0.816488
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,0.133921,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861,H,-0.540815,1.447527,-0.876644,-0.012689,1.085797,0.008001,4,0.133923,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579,-0.542965,1.453558,...,1.783147,0,1.091953,-0.012698,1.085804,0.008001,1.783157,0,1.091946,-0.012698,1.085804,0.008001,1.783158,4,1.783157,-0.523814,1.437933,0.906397,1.091953,2,1.783158,1.011731,1.463751,0.000277,1.091946,1.091945,1.091945,1.091953,1.091946,1.783157,1.783158,-0.333335,-0.000016,-0.333338,-0.816498,0.816496,-0.500002,0.500018,-0.816496,0.816505
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,0.133921,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861,H,-0.523814,1.437933,0.906397,-0.012689,1.085797,0.008001,4,0.133923,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013,-0.525964,1.443964,...,1.783157,0,1.091953,-0.012698,1.085804,0.008001,1.783157,0,1.091948,-0.012698,1.085804,0.008001,1.783157,4,1.783157,-0.523814,1.437933,0.906397,1.091953,1,1.783157,0.002150,-0.006031,0.001976,1.091948,1.091945,1.091945,1.091953,1.091948,1.783157,1.783157,-0.333347,-1.000000,-0.333351,-0.816502,0.816500,-1.000000,1.000000,-0.816500,0.816509
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,0.133922,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967,C,-0.012698,1.085804,0.008001,-0.012689,1.085797,0.008001,4,-0.535689,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170,-1.024429,-0.377947,...,1.091952,0,1.091952,-0.012698,1.085804,0.008001,1.783158,3,1.091946,-0.540815,1.447527,-0.876644,1.091953,3,1.783158,-0.540815,1.447527,-0.876644,1.091952,1,1.091953,0.002150,-0.006031,0.001976,1.091946,1.091946,1.091946,1.091952,1.091946,1.783158,1.091953,0.333352,-0.000028,-0.

## Create Neural Network Model
I use the additional data (mulliken charge and tensor) as the output of the middle of neural network model. I am too lazy to make the pipeline model. I hope this works as the kind of attractors.*(I don't know how to call this, so let it call as attractors in this kernel)*

In [11]:
def create_nn_model(input_shape):
    inp = Input(shape=(input_shape,))
    x = Dense(256, activation="relu")(inp)
    x = BatchNormalization()(x)
    x = Dense(256, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dense(128, activation="relu")(x)
    x = BatchNormalization()(x)
    out1 = Dense(2, activation="linear")(x)#mulliken charge 2
    out2 = Dense(6, activation="linear")(x)#tensor 6(xx,yy,zz)
    out3 = Dense(12, activation="linear")(x)#tensor 12(others) 
    x = Dense(64, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dense(64, activation="relu")(x)
    x = BatchNormalization()(x)
    out = Dense(1, activation="linear")(x)#scalar_coupling_constant    
    model = Model(inputs=inp, outputs=[out,out1,out2,out3])
    return model

create neural networks for each molecule type. (calculate 8 models in total) 

In [12]:
mol_types=df_train["type"].unique()
cv_score=[]
cv_score_total=0
test_prediction=np.zeros(len(df_test))

for mol_type in mol_types:
    df_train_=df_train[df_train["type"]==mol_type]
    df_test_=df_test[df_test["type"]==mol_type]
    input_features=["x_0","y_0","z_0","x_1","y_1","z_1","c_x","c_y","c_z",
                    'x_closest_0','y_closest_0','z_closest_0','x_closest_1','y_closest_1','z_closest_1',
                    "distance","distance_center0","distance_center1","distance_c0","distance_c1","distance_f0","distance_f1",
                    "cos_c0_c1","cos_f0_f1","cos_center0_center1","cos_c0","cos_c1","cos_f0","cos_f1","cos_center0","cos_center1",
                    "atom_n"
                   ]
    input_data=StandardScaler().fit_transform(pd.concat([df_train_.loc[:,input_features],df_test_.loc[:,input_features]]))
    target_data=df_train_.loc[:,"scalar_coupling_constant"].values
    target_data_1=df_train_.loc[:,["charge_0","charge_1"]]
    target_data_2=df_train_.loc[:,["XX_0","YY_0","ZZ_0","XX_1","YY_1","ZZ_1"]]
    target_data_3=df_train_.loc[:,["YX_0","ZX_0","XY_0","ZY_0","XZ_0","YZ_0","YX_1","ZX_1","XY_1","ZY_1","XZ_1","YZ_1"]]
    
    #following parameters should be adjusted to control the loss function
    #if all parameters are zero, attractors do not work. (-> simple neural network)
    m1=1
    m2=4
    m3=1
    target_data_1=m1*(StandardScaler().fit_transform(target_data_1))
    target_data_2=m2*(StandardScaler().fit_transform(target_data_2))
    target_data_3=m3*(StandardScaler().fit_transform(target_data_3))
    
    train_index, cv_index = train_test_split(np.arange(len(df_train_)),random_state=111, test_size=0.2)
    
    train_input=input_data[train_index]
    cv_input=input_data[cv_index]
    train_target=target_data[train_index]
    cv_target=target_data[cv_index]
    train_target_1=target_data_1[train_index]
    cv_target_1=target_data_1[cv_index]
    train_target_2=target_data_2[train_index]
    cv_target_2=target_data_2[cv_index]
    train_target_3=target_data_3[train_index]
    cv_target_3=target_data_3[cv_index]
    test_input=input_data[len(df_train_):,:]

    epoch_n=12#as trial. Longer epoch improve the score
    verbose=1
    nn_model=create_nn_model(train_input.shape[1])
    nn_model.compile(loss='mae', optimizer=Adam(lr=1e-3))#, metrics=[auc])
    es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0., patience=12,#val_auc
                                 verbose=verbose, mode='min', baseline=None, restore_best_weights=True)
    rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                                      patience=4, min_lr=2e-6, mode='min', verbose=verbose)
    nn_model.fit(train_input,[train_target,train_target_1,train_target_2,train_target_3], 
            validation_data=(cv_input,[cv_target,cv_target_1,cv_target_2,cv_target_3]), 
            callbacks=[es, rlr], epochs=epoch_n, batch_size=64, verbose=verbose)
    cv_predict=nn_model.predict(cv_input)
    
    accuracy=np.mean(np.abs(cv_target-cv_predict[0][:,0]))
    cv_score.append(np.log(accuracy))
    cv_score_total+=np.log(accuracy)
    #print(mol_type,": log score is ",np.log(accuracy))    
    test_predict=nn_model.predict(test_input)
    test_prediction[df_test["type"]==mol_type]=test_predict[0][:,0]
    K.clear_session()

cv_score_total/=len(mol_types)
#print("total score is",cv_score_total)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 567536 samples, validate on 141885 samples
Epoch 1/10
567536/567536 [==============================] - 68s 119us/step - loss: 12.1848 - dense_10_loss: 8.9959 - dense_5_loss: 0.4113 - dense_6_loss: 2.0541 - dense_7_loss: 0.7234 - val_loss: 6.0881 - val_dense_10_loss: 3.1220 - val_dense_5_loss: 0.3556 - val_dense_6_loss: 1.8933 - val_dense_7_loss: 0.7172
Epoch 2/10
567536/567536 [==============================] - 63s 110us/step - loss: 6.6334 - dense_10_loss: 3.6353 - dense_5_loss: 0.3660 - dense_6_loss: 1.9162 - dense_7_loss: 0.7159 - val_loss: 5.8269 - val_dense_10_loss: 2.9411 - val_dense_5_loss: 0.3398 - val_dense_6_loss: 1.8296 - val_dense_7_loss: 0.7164
Epoch 3/10
567536/567536 [==============================] - 65s 115us/step - loss: 6.3914 - dense_10_loss: 3.4618 - dense_5_loss: 0.3512 - dense_6_loss: 1.8629 - dense_7_loss: 0.7155 - val_loss: 5.6961 - v

34690/34690 [==============================] - 4s 104us/step - loss: 4.5835 - dense_10_loss: 1.9424 - dense_5_loss: 0.2988 - dense_6_loss: 1.6618 - dense_7_loss: 0.6805 - val_loss: 4.0778 - val_dense_10_loss: 1.5491 - val_dense_5_loss: 0.2806 - val_dense_6_loss: 1.5726 - val_dense_7_loss: 0.6756
Epoch 6/10
34690/34690 [==============================] - 4s 106us/step - loss: 4.4759 - dense_10_loss: 1.8854 - dense_5_loss: 0.2916 - dense_6_loss: 1.6200 - dense_7_loss: 0.6789 - val_loss: 4.0056 - val_dense_10_loss: 1.5195 - val_dense_5_loss: 0.2582 - val_dense_6_loss: 1.5523 - val_dense_7_loss: 0.6756
Epoch 7/10
34690/34690 [==============================] - 4s 109us/step - loss: 4.3782 - dense_10_loss: 1.8308 - dense_5_loss: 0.2826 - dense_6_loss: 1.5878 - dense_7_loss: 0.6770 - val_loss: 3.8475 - val_dense_10_loss: 1.4015 - val_dense_5_loss: 0.2552 - val_dense_6_loss: 1.5179 - val_dense_7_loss: 0.6729
Epoch 8/10
34690/34690 [==============================] - 4s 107us/step - loss: 4.3251 

472493/472493 [==============================] - 55s 117us/step - loss: 4.5510 - dense_10_loss: 0.9909 - dense_5_loss: 0.3064 - dense_6_loss: 2.4899 - dense_7_loss: 0.7638 - val_loss: 3.9175 - val_dense_10_loss: 0.6516 - val_dense_5_loss: 0.2316 - val_dense_6_loss: 2.2785 - val_dense_7_loss: 0.7557
Epoch 2/10
472493/472493 [==============================] - 53s 113us/step - loss: 4.0124 - dense_10_loss: 0.7364 - dense_5_loss: 0.2512 - dense_6_loss: 2.2701 - dense_7_loss: 0.7547 - val_loss: 3.7149 - val_dense_10_loss: 0.5638 - val_dense_5_loss: 0.2111 - val_dense_6_loss: 2.1865 - val_dense_7_loss: 0.7536
Epoch 3/10
472493/472493 [==============================] - 52s 110us/step - loss: 3.8690 - dense_10_loss: 0.6865 - dense_5_loss: 0.2389 - dense_6_loss: 2.1895 - dense_7_loss: 0.7542 - val_loss: 3.5873 - val_dense_10_loss: 0.5285 - val_dense_5_loss: 0.2011 - val_dense_6_loss: 2.1038 - val_dense_7_loss: 0.7539
Epoch 4/10
472493/472493 [==============================] - 52s 110us/step - l

Epoch 7/10
133136/133136 [==============================] - 14s 103us/step - loss: 2.9503 - dense_10_loss: 0.3461 - dense_5_loss: 0.2532 - dense_6_loss: 1.6531 - dense_7_loss: 0.6978 - val_loss: 2.7823 - val_dense_10_loss: 0.2904 - val_dense_5_loss: 0.2186 - val_dense_6_loss: 1.5756 - val_dense_7_loss: 0.6977
Epoch 8/10
133136/133136 [==============================] - 14s 106us/step - loss: 2.9124 - dense_10_loss: 0.3369 - dense_5_loss: 0.2482 - dense_6_loss: 1.6295 - dense_7_loss: 0.6977 - val_loss: 2.7600 - val_dense_10_loss: 0.2836 - val_dense_5_loss: 0.2173 - val_dense_6_loss: 1.5608 - val_dense_7_loss: 0.6983
Epoch 9/10
133136/133136 [==============================] - 14s 107us/step - loss: 2.8799 - dense_10_loss: 0.3306 - dense_5_loss: 0.2455 - dense_6_loss: 1.6063 - dense_7_loss: 0.6975 - val_loss: 2.7320 - val_dense_10_loss: 0.2806 - val_dense_5_loss: 0.2119 - val_dense_6_loss: 1.5415 - val_dense_7_loss: 0.6980
Epoch 10/10
133136/133136 [==============================] - 15s 11

If you need higher score, you can increase the number of epochs and adjust the hyper parameters.

In [13]:
i=0
for mol_type in mol_types: 
    print(mol_type,": cv score is ",cv_score[i])
    i+=1
print("total cv score is",cv_score_total)

1JHC : cv score is  0.8916530837139433
2JHH : cv score is  -0.8149072033859551
1JHN : cv score is  0.23093738254476512
2JHN : cv score is  -0.9596804349972338
2JHC : cv score is  -0.2595848701337643
3JHH : cv score is  -0.805650843151497
3JHC : cv score is  -0.370646211453285
3JHN : cv score is  -1.265383915345859
total cv score is -0.41915787652611075


In [14]:
def submit(predictions):
    submit = pd.read_csv('../input/sample_submission.csv')
    submit["scalar_coupling_constant"] = predictions
    submit.to_csv("submission.csv", index=False)

submit(test_prediction)

## Check the effect of middle outputs
Now, let's compare with the model without attractors by changing parameter m1,m2,m3 to zero.

In [15]:
mol_types=df_train["type"].unique()
cv_score=[]
cv_score_total=0
#test_prediction=np.zeros(len(df_test))

for mol_type in mol_types:
    df_train_=df_train[df_train["type"]==mol_type]
    df_test_=df_test[df_test["type"]==mol_type]
    input_features=["x_0","y_0","z_0","x_1","y_1","z_1","c_x","c_y","c_z",
                    'x_closest_0','y_closest_0','z_closest_0','x_closest_1','y_closest_1','z_closest_1',
                    "distance","distance_center0","distance_center1","distance_c0","distance_c1","distance_f0","distance_f1",
                    "cos_c0_c1","cos_f0_f1","cos_center0_center1","cos_c0","cos_c1","cos_f0","cos_f1","cos_center0","cos_center1",
                    "atom_n"
                   ]
    input_data=StandardScaler().fit_transform(pd.concat([df_train_.loc[:,input_features],df_test_.loc[:,input_features]]))
    target_data=df_train_.loc[:,"scalar_coupling_constant"].values
    target_data_1=df_train_.loc[:,["charge_0","charge_1"]]
    target_data_2=df_train_.loc[:,["XX_0","YY_0","ZZ_0","XX_1","YY_1","ZZ_1"]]
    target_data_3=df_train_.loc[:,["YX_0","ZX_0","XY_0","ZY_0","XZ_0","YZ_0","YX_1","ZX_1","XY_1","ZY_1","XZ_1","YZ_1"]]
    
    #following parameters should be adjusted to control the loss function
    #if all parameters are zero, attractors do not work. (-> simple neural network)
    m1=0
    m2=0
    m3=0
    target_data_1=m1*(StandardScaler().fit_transform(target_data_1))
    target_data_2=m2*(StandardScaler().fit_transform(target_data_2))
    target_data_3=m3*(StandardScaler().fit_transform(target_data_3))
    
    train_index, cv_index = train_test_split(np.arange(len(df_train_)),random_state=111, test_size=0.2)
    
    train_input=input_data[train_index]
    cv_input=input_data[cv_index]
    train_target=target_data[train_index]
    cv_target=target_data[cv_index]
    train_target_1=target_data_1[train_index]
    cv_target_1=target_data_1[cv_index]
    train_target_2=target_data_2[train_index]
    cv_target_2=target_data_2[cv_index]
    train_target_3=target_data_3[train_index]
    cv_target_3=target_data_3[cv_index]
    test_input=input_data[len(df_train_):,:]

    epoch_n=12#for trial. Longer epoch improve the score
    verbose=1
    nn_model=create_nn_model(train_input.shape[1])
    nn_model.compile(loss='mae', optimizer=Adam(lr=1e-3))#, metrics=[auc])
    es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0., patience=12,#val_auc
                                 verbose=verbose, mode='min', baseline=None, restore_best_weights=True)
    rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                                      patience=4, min_lr=2e-6, mode='min', verbose=verbose)
    nn_model.fit(train_input,[train_target,train_target_1,train_target_2,train_target_3], 
            validation_data=(cv_input,[cv_target,cv_target_1,cv_target_2,cv_target_3]), 
            callbacks=[es, rlr], epochs=epoch_n, batch_size=64, verbose=verbose)
    cv_predict=nn_model.predict(cv_input)
    
    accuracy=np.mean(np.abs(cv_target-cv_predict[0][:,0]))
    cv_score.append(np.log(accuracy))
    cv_score_total+=np.log(accuracy)
    #print(mol_type,": log score is ",np.log(accuracy))    
    #test_predict=nn_model.predict(test_input)
    #test_prediction[df_test["type"]==mol_type]=test_predict[0][:,0]
    K.clear_session()

cv_score_total/=len(mol_types)
#print("total score is",cv_score_total)

Train on 567536 samples, validate on 141885 samples
Epoch 1/10
567536/567536 [==============================] - 61s 107us/step - loss: 9.3691 - dense_10_loss: 9.3151 - dense_5_loss: 0.0171 - dense_6_loss: 0.0176 - dense_7_loss: 0.0193 - val_loss: 3.5532 - val_dense_10_loss: 3.5285 - val_dense_5_loss: 0.0085 - val_dense_6_loss: 0.0085 - val_dense_7_loss: 0.0077
Epoch 2/10
567536/567536 [==============================] - 61s 107us/step - loss: 3.8390 - dense_10_loss: 3.8183 - dense_5_loss: 0.0069 - dense_6_loss: 0.0069 - dense_7_loss: 0.0070 - val_loss: 3.4073 - val_dense_10_loss: 3.3854 - val_dense_5_loss: 0.0065 - val_dense_6_loss: 0.0083 - val_dense_7_loss: 0.0071
Epoch 3/10
567536/567536 [==============================] - 60s 106us/step - loss: 3.6611 - dense_10_loss: 3.6452 - dense_5_loss: 0.0055 - dense_6_loss: 0.0053 - dense_7_loss: 0.0052 - val_loss: 3.4060 - val_dense_10_loss: 3.3910 - val_dense_5_loss: 0.0062 - val_dense_6_loss: 0.0043 - val_dense_7_loss: 0.0044
Epoch 4/10
5675

KeyboardInterrupt: 

In [ ]:
i=0
for mol_type in mol_types: 
    print(mol_type,": cv score is ",cv_score[i])
    i+=1
print("total cv score is",cv_score_total)

The score with attractors is slightly better than without attractors.